In [5]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from PIL import Image, ImageTk
from collections import deque 

In [ ]:
class ImageEnhancerApp:
    def __init__(self, master):
        self.master = master
        master.title("Image Enhancer")
        master.configure(bg="#D3D3D3")  # Set background color

        # Set font and styles
        self.title_font = ("Times New Roman", 16, "bold")
        self.label_font = ("Helvetica", 12)
        self.button_font = ("Helvetica", 10, "bold")

        # Create main frame
        self.main_frame = tk.Frame(master, bg="#D3D3D3")
        self.main_frame.pack()
        
        # Display application name
        self.app_name_label = tk.Label(self.main_frame, text="Chroma Image Boost", font=self.title_font, bg="#00FF00")
        self.app_name_label.grid(row=0, column=0, columnspan=5, padx=10, pady=10, sticky="n")

        self.image_label = tk.Label(self.main_frame, bg="#D3D3D3")
        self.image_label.grid(row=2, column=0, columnspan=5, padx=10, pady=10)

        self.brightness_scale = tk.Scale(self.main_frame, label="Brightness", from_=-100, to=100, orient=tk.HORIZONTAL, bg="#f0f0f0", font=self.label_font)
        self.brightness_scale.set(0)  # Default value
        self.brightness_scale.grid(row=3, column=0, padx=10, pady=5, sticky="ew")

        self.contrast_scale = tk.Scale(self.main_frame, label="Contrast", from_=-100, to=100, orient=tk.HORIZONTAL, bg="#f0f0f0", font=self.label_font)
        self.contrast_scale.set(0)  # Default value
        self.contrast_scale.grid(row=3, column=1, padx=10, pady=5, sticky="ew")

        self.sharpness_scale = tk.Scale(self.main_frame, label="Sharpness", from_=0, to=2, resolution=0.1, orient=tk.HORIZONTAL, bg="#f0f0f0", font=self.label_font)
        self.sharpness_scale.set(1)  # Default value
        self.sharpness_scale.grid(row=3, column=2, padx=10, pady=5, sticky="ew")

        self.saturation_scale = tk.Scale(self.main_frame, label="Saturation", from_=0, to=2, resolution=0.1, orient=tk.HORIZONTAL, bg="#f0f0f0", font=self.label_font)
        self.saturation_scale.set(1)  # Default value
        self.saturation_scale.grid(row=3, column=3, padx=10, pady=5, sticky="ew")

        self.enhance_button = tk.Button(self.main_frame, text="Enhance Image", command=self.enhance_image, bg="#00FF00", fg="#000000", font=self.button_font)
        self.enhance_button.grid(row=6, column=2, padx=10, pady=5, sticky="ew")

        self.load_button = tk.Button(self.main_frame, text="Load Image", command=self.load_image, bg="#008CBA", fg="#000000", font=self.button_font)
        self.load_button.grid(row=1, column=1, columnspan=2, padx=10, pady=5, sticky="ew")

        self.save_button = tk.Button(self.main_frame, text="Save Image", command=self.save_image, bg="#40E0D0", fg="#000000", font=self.button_font)
        self.save_button.grid(row=6, column=3, padx=10, pady=5, sticky="ew")

        self.close_button = tk.Button(self.main_frame, text="Close Window", command=master.destroy, bg="#FF0000", fg="#000000", font=self.button_font)
        self.close_button.grid(row=8, column=3, padx=10, pady=5, sticky="ew")
        
        self.reset_button = tk.Button(self.main_frame, text="Reset Image", command=self.reset_image, bg="#EE82EE", fg="#000000", font=self.button_font)
        self.reset_button.grid(row=6, column=1, padx=10, pady=5, sticky="ew")

        self.rotate_button = tk.Button(self.main_frame, text="Rotate Image", command=self.rotate_image, bg="#9ACD32", fg="#000000", font=self.button_font)
        self.rotate_button.grid(row=8, column=0, padx=10, pady=5, sticky="ew")
        
        self.undo_button = tk.Button(self.main_frame, text="Undo", command=self.undo_image, bg="#708090", fg="#FFFFFF", font=self.button_font, width=10, height=1)
        self.undo_button.grid(row=0, column=0, padx=10, pady=5, sticky="ew")

        self.redo_button = tk.Button(self.main_frame, text="Redo", command=self.redo_image, bg="#708090", fg="#FFFFFF", font=self.button_font, width=10, height=1)
        self.redo_button.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

        self.zoom_in_button = tk.Button(self.main_frame, text="Zoom In", command=self.zoom_in, bg="#FFFF00", fg="#000000", font=self.button_font)
        self.zoom_in_button.grid(row=7, column=0, padx=10, pady=5, sticky="ew")

        self.zoom_out_button = tk.Button(self.main_frame, text="Zoom Out", command=self.zoom_out, bg="#FFFF00", fg="#000000", font=self.button_font)
        self.zoom_out_button.grid(row=7, column=2, padx=10, pady=5, sticky="ew")

        self.info_button = tk.Button(self.main_frame, text="Image Info", command=self.image_info, bg="#FFE4B5", fg="#000000", font=self.button_font)
        self.info_button.grid(row=7, column=1, padx=10, pady=5, sticky="ew")
        
        self.crop_button = tk.Button(self.main_frame, text="Crop Image", command=self.crop_image, bg="#DEB887",fg="#000000", font=self.button_font)
        self.crop_button.grid(row=6, column=0, padx=10, pady=5, sticky="ew")
        

        # Set initial state
        self.original_image = None
        self.adjusted_image = None
        self.undo_stack = deque(maxlen=10)
        self.redo_stack = deque(maxlen=10)
        self.zoom_level = 1.0
        self.crop_mode = False
        self.crop_start_x = 0
        self.crop_start_y = 0
        self.crop_end_x = 0
        self.crop_end_y = 0
        self.crop_frame = None
        self.crop_guide = None
        

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.original_image = cv2.imread(file_path)
            self.cropped_image = self.original_image.copy()  # Initialize cropped image as the original
            self.display_image(self.original_image)
                   

    def enhance_image(self):
        if self.original_image is not None:
            brightness = self.brightness_scale.get()
            contrast = self.contrast_scale.get()
            sharpness = self.sharpness_scale.get()
            saturation = self.saturation_scale.get()

            self.adjusted_image = self.adjust_image(self.original_image, brightness, contrast, sharpness, saturation)
            self.display_image(self.adjusted_image)
            # Clear redo stack whenever new adjustment is made
            self.redo_stack.clear()
            # Add the original image to the undo stack
            self.undo_stack.append(self.original_image.copy())

    def save_image(self):
        if self.adjusted_image is not None:
            file_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg"), ("All files", "*.*")])
            if file_path:
                cv2.imwrite(file_path, self.adjusted_image)

    def reset_image(self):
        if self.original_image is not None:
            self.display_image(self.original_image)
            self.adjusted_image = None
            # Reset sliders to their default positions
            self.brightness_scale.set(0)
            self.contrast_scale.set(0)
            self.sharpness_scale.set(1)
            self.saturation_scale.set(1)
            # Clear undo and redo stacks
            self.undo_stack.clear()
            self.redo_stack.clear()
            # Remove crop frame and guide
            if hasattr(self, 'crop_frame'):
                self.crop_frame.destroy()
                del self.crop_frame
            if hasattr(self, 'crop_guide'):
                self.crop_guide = None
    def rotate_image(self):
        if self.original_image is not None:
            angle = 90  # Change the angle as per requirement
            rotated_image = cv2.rotate(self.original_image, cv2.ROTATE_90_CLOCKWISE)
            self.original_image = rotated_image
            if self.adjusted_image is not None:
                self.adjusted_image = cv2.rotate(self.adjusted_image, cv2.ROTATE_90_CLOCKWISE)
            self.display_image(rotated_image)
            # Clear undo and redo stacks as rotation is not reversible
            self.undo_stack.clear()
            self.redo_stack.clear()
    
    def undo_image(self):
        if self.undo_stack:
            last_state = self.undo_stack.pop()
            self.redo_stack.append(self.adjusted_image)
            self.adjusted_image = last_state
            self.display_image(self.adjusted_image)

    def redo_image(self):
        if self.redo_stack:
            last_state = self.redo_stack.pop()
            self.undo_stack.append(self.adjusted_image)
            self.adjusted_image = last_state
            self.display_image(self.adjusted_image)


    def zoom_in(self):
        if self.zoom_level < 2.0:  # Limit maximum zoom level
            self.zoom_level += 0.1
            self.display_image(self.original_image)

    def zoom_out(self):
        if self.zoom_level > 0.1:  # Limit minimum zoom level
            self.zoom_level -= 0.1
            self.display_image(self.original_image)

    def image_info(self):
        if self.original_image is not None:
            height, width, _ = self.original_image.shape
            info_str = f"Width: {width}, Height: {height}"
            messagebox.showinfo("Image Info", info_str)
       
                
    def display_image(self, image):
        if isinstance(image, np.ndarray):
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)

        # Resize image to fit within a specific frame (e.g., 600x400)
        width, height = 600, 400
        new_width = int(width * self.zoom_level)
        new_height = int(height * self.zoom_level)
        image = image.resize((new_width, new_height), Image.LANCZOS)

        image = ImageTk.PhotoImage(image)
        self.image_label.configure(image=image)
        self.image_label.image = image
        

    def adjust_image(self, image, brightness, contrast, sharpness, saturation):
        # Apply brightness and contrast adjustment
        alpha = (float(contrast) + 100) / 100
        beta = float(brightness)
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

        # Apply sharpness
        blurred = cv2.GaussianBlur(adjusted_image, (0, 0), sharpness)
        sharpened = cv2.addWeighted(adjusted_image, 1.5, blurred, -0.5, 0)

        # Apply saturation
        hsv = cv2.cvtColor(sharpened, cv2.COLOR_BGR2HSV).astype(float)
        hsv[:, :, 1] *= saturation
        hsv[:, :, 1] = np.clip(hsv[:, :, 1], 0, 255)
        adjusted_image = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

        return adjusted_image
    
    def crop_image(self):
        if self.original_image is not None:
            messagebox.showinfo("Crop Image", "Click and drag to select the region to crop.")
            self.crop_mode = True
            self.crop_start_x, self.crop_start_y = 0, 0
            self.crop_end_x, self.crop_end_y = 0, 0
            self.crop_frame = tk.Canvas(self.image_label, bg="White", bd=2, highlightbackground="black", highlightthickness=1)
            self.crop_frame.bind("<Button-1>", self.start_crop)
            self.crop_frame.bind("<B1-Motion>", self.adjust_crop_frame)
            self.crop_frame.bind("<ButtonRelease-1>", self.release_crop_frame)
            self.crop_frame.place(relx=0, rely=0, relwidth=1, relheight=1)
            self.crop_guide = self.crop_frame.create_rectangle(0, 0, 0, 0, outline="black", dash=(2, 2))

    def start_crop(self, event):
        self.crop_start_x = event.x
        self.crop_start_y = event.y

    def adjust_crop_frame(self, event):
        self.crop_end_x = event.x
        self.crop_end_y = event.y
        self.crop_frame.coords(self.crop_guide, self.crop_start_x, self.crop_start_y, self.crop_end_x, self.crop_end_y)

    def release_crop_frame(self, event):
        # Get the dimensions of the original image
        image_width = self.original_image.shape[1]
        image_height = self.original_image.shape[0]

        # Calculate the crop coordinates relative to the original image
        x1 = min(self.crop_start_x, self.crop_end_x) * image_width / self.image_label.winfo_width()
        y1 = min(self.crop_start_y, self.crop_end_y) * image_height / self.image_label.winfo_height()
        x2 = max(self.crop_start_x, self.crop_end_x) * image_width / self.image_label.winfo_width()
        y2 = max(self.crop_start_y, self.crop_end_y) * image_height / self.image_label.winfo_height()

        # Crop the image based on the calculated coordinates
        cropped_image = self.original_image[int(y1):int(y2), int(x1):int(x2)].copy()
        self.display_image(cropped_image)
        self.crop_mode = False



def main():
    root = tk.Tk()
    root.attributes('-fullscreen', True)  # Open in full-screen mode
    app = ImageEnhancerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
